In [47]:
import pandas as pd
import numpy as np
import torch 
from torch import optim
from torch import nn
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchinfo import summary
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from torch.utils.data import DataLoader, TensorDataset

In [48]:
device = 'mps' if torch.mps.is_available() else 'cpu'
print(device)
df = pd.read_csv('train.csv',index_col='PassengerId')
df.head()

mps


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [50]:
from sklearn.preprocessing import StandardScaler
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if big_string.find(substring) != -1:  # Corrected the function call
            return substring
    print(big_string)  # Use parentheses for print in Python 3
    return np.nan  # Return np.nan to indicate no match

def replace_titles(x):
    title=x['Title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
        return 'Mr'
    elif title in ['Countess', 'Mme']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title


def preprocess_dataset(df : pd.DataFrame):
    title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
                    'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                    'Don', 'Jonkheer']
    df['Title']=df['Name'].map(lambda x: substrings_in_string(x, title_list))
    df['Title']=df.apply(replace_titles, axis=1)
    
    df = df.drop(columns=['Name', 'Cabin', 'Ticket'], axis=1)

    # One-hot Encoding
    df = pd.get_dummies(df, drop_first=True, dtype=np.int32)

    # Handle missing values
    df["Age"].fillna(df["Age"].median(), inplace=True)  # Fill missing ages with median
    df["Fare"].fillna(df["Fare"].median(), inplace=True)  # Fill missing fares with median

    # Feature Engineering
    df['Family_Size']=df['SibSp']+df['Parch'] + 1
    df['Age*Fare']=df['Age']*df['Fare']

    
    return df


In [51]:
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [52]:


# Apply standardization only on the numerical columns



In [53]:
trainingCutoff = int(len(df))
numerical_columns = ['Age','Fare','Age*Fare']
scaler = StandardScaler()

df_train = df
df_train = preprocess_dataset(df_train)
df_train[numerical_columns] = scaler.fit_transform(df_train[numerical_columns])



print(f'Length of train: {len(df_train)}')



Length of train: 891


/var/folders/c_/jxz3szl16zbcx94x257l4skc0000gn/T/ipykernel_92129/3613148263.py:39: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Age"].fillna(df["Age"].median(), inplace=True)  # Fill missing ages with median
/var/folders/c_/jxz3szl16zbcx94x257l4skc0000gn/T/ipykernel_92129/3613148263.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate objec

In [54]:
df_val = df[trainingCutoff:]
df_val = preprocess_dataset(df_val)
#df_val[numerical_columns] = scaler.transform(df_val[numerical_columns])
print(f'Length of validation: {len(df_val)}')

Length of validation: 0


/var/folders/c_/jxz3szl16zbcx94x257l4skc0000gn/T/ipykernel_92129/3613148263.py:39: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Age"].fillna(df["Age"].median(), inplace=True)  # Fill missing ages with median
/var/folders/c_/jxz3szl16zbcx94x257l4skc0000gn/T/ipykernel_92129/3613148263.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate objec

In [55]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Survived     891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Age          891 non-null    float64
 3   SibSp        891 non-null    int64  
 4   Parch        891 non-null    int64  
 5   Fare         891 non-null    float64
 6   Sex_male     891 non-null    int32  
 7   Embarked_Q   891 non-null    int32  
 8   Embarked_S   891 non-null    int32  
 9   Title_Miss   891 non-null    int32  
 10  Title_Mr     891 non-null    int32  
 11  Title_Mrs    891 non-null    int32  
 12  Family_Size  891 non-null    int64  
 13  Age*Fare     891 non-null    float64
dtypes: float64(3), int32(6), int64(5)
memory usage: 83.5 KB


In [56]:
BATCH_SIZE = 64

class CustomDataset(Dataset):
    def __init__(self, dataframe, target_column, transform=None):
        self.dataframe = dataframe
        self.target_column = target_column
        self.transform = transform

    def __len__(self):
        # Return the total number of samples
        return len(self.dataframe)

    def __getitem__(self, index):
        # Retrieve a single sample from the dataframe
        row = self.dataframe.iloc[index]
        
        # Features are all columns except the target_column
        X = row.drop(self.target_column).values.astype(np.float32)
        y = row[self.target_column]
        
        # Convert to tensors
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.tensor(y, dtype=torch.float32)

        # Apply any transformations (if needed)
        if self.transform:
            X_tensor = self.transform(X_tensor)
        
        return X_tensor, y_tensor

In [57]:
train_dataset = CustomDataset(dataframe=df_train,target_column='Survived')
validation_dataset = CustomDataset(dataframe=df_val, target_column='Survived')

In [58]:
# Create DataLoader for training and testing
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [59]:
x, y = next(iter(train_loader))
print(f'X (Shape): {x.shape} | Y (Shape): {y.shape}')
print(f'Sample X: {x[0]} \n Sample Y: {y[0]}')

X (Shape): torch.Size([64, 13]) | Y (Shape): torch.Size([64])
Sample X: tensor([ 3.0000, -1.0268,  5.0000,  2.0000,  0.2959,  0.0000,  0.0000,  1.0000,
         1.0000,  0.0000,  0.0000,  8.0000, -0.1432]) 
 Sample Y: 0.0


In [ ]:
torch.manual_seed(42)
class BaseModel(nn.Module):
    def __init__(self, in_features, hidden_units, out_features=1):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(in_features, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, out_features),
            nn.ReLU(),
            nn.Linear(hidden_units, out_features),
            nn.ReLU(),
            nn.Linear(hidden_units, out_features),
            nn.ReLU(),
            nn.Linear(hidden_units, out_features),
            )

    def forward(self, x):
        return self.layers(x)
        
        
    

In [66]:
torch.manual_seed(42)
model = BaseModel(in_features=x.shape[1], hidden_units=64, out_features=1)

    

In [67]:
summary(model=model, input_size=(BATCH_SIZE, x.shape[1]))

Layer (type:depth-idx)                   Output Shape              Param #
BaseModel                                [64, 1]                   --
├─Sequential: 1-1                        [64, 1]                   --
│    └─Linear: 2-1                       [64, 64]                  896
│    └─ReLU: 2-2                         [64, 64]                  --
│    └─Linear: 2-3                       [64, 128]                 8,320
│    └─ReLU: 2-4                         [64, 128]                 --
│    └─Linear: 2-5                       [64, 64]                  8,256
│    └─ReLU: 2-6                         [64, 64]                  --
│    └─Linear: 2-7                       [64, 1]                   65
Total params: 17,537
Trainable params: 17,537
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 1.12
Input size (MB): 0.00
Forward/backward pass size (MB): 0.13
Params size (MB): 0.07
Estimated Total Size (MB): 0.21

In [68]:
survived_counts = df_train['Survived'].value_counts()
pos_weight = survived_counts[0] / survived_counts[1]  # Weight for the positive class
survived_counts

Survived
0    549
1    342
Name: count, dtype: int64

In [69]:
loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weight, dtype=torch.float32))  # Binary Cross Entropy Loss
optimizer = optim.Adam(params=model.parameters(), lr=0.0001)

In [70]:
from tqdm import tqdm
def train_step(model:nn.Module ,train_loader: torch.utils.data.DataLoader, optimizer: torch.optim, loss_fn: nn.Module):
#    print('TRAINING:')
    model = model.train()
    # Forward Pass
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(train_loader):
        
        X, y = X.to(device), y.to(device)
        # Forward Pass
        y_logits = model(X)
        # Calculate Accuracy
        y_prob = torch.sigmoid(y_logits)
        
        # Calculate the Loss
        loss = loss_fn(y_logits.squeeze(), y)
        train_loss += loss.item()
        
        # Optimizer Zero Grad
        optimizer.zero_grad()

        # Backpropagation
        loss.backward()

        # Optimizer Step
        optimizer.step()
        
        y_pred = (y_prob >= 0.5).float()
        #print(f'y Shape: {y.shape} | y_logits Shape: {y_logits.shape} | y_prob Shape: {y_prob.shape} | y_pred Shape: {y_pred.shape}')

        acc = accuracy_score(y_true=y.cpu().numpy(), y_pred=y_pred.cpu().numpy())
        train_acc += acc
    
    train_loss /= len(train_loader)
    train_acc /= len(train_loader)

    return train_loss, train_acc
    
def val_step(model, val_loader, loss_fn):
    #print('VALIDATION:')
    model = model.eval()
    val_loss ,val_acc = 0, 0

    with torch.inference_mode():
        for batch, (X, y) in enumerate(val_loader):
            X, y = X.to(device), y.to(device)
            
            y_logits = model(X)
            y_probs = torch.sigmoid(y_logits)
            
            y_pred = (y_probs >= 0.5).float()

            val_loss += loss_fn(y_logits.squeeze(), y).item()
            val_acc += accuracy_score(y_true=y.cpu().numpy(), y_pred=y_pred.cpu().numpy())
            #print(f'y Shape: {y.shape} | y_logits Shape: {y_logits.shape} | y_prob Shape: {y_probs.shape} | y_pred Shape: {y_pred.shape}')
        
    val_loss /= len(val_loader)
    val_acc /= len(val_loader)
        
    return val_loss, val_acc
        

def train_model(epochs:int, model:nn.Module, optimizer: torch.optim, loss_fn, train_dataloader
                , val_dataloader):
    evaluation_results = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    model = model.to(device)
    for epoch in tqdm(range(1, epochs + 1)):
        
        print(f'Epoch {epoch}\n----------------------')
        
        train_loss, train_acc = train_step(
            model=model, 
            train_loader=train_dataloader,
            optimizer=optimizer, 
            loss_fn=loss_fn,
            )
        
      # val_loss, val_acc = val_step(
#     model=model, 
       #     val_loader=val_dataloader,
        #    loss_fn=loss_fn,
         #   )
        
        print(f'Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.4f} ')
        #print(f'Valdation Loss: {val_loss:.4f} | Validation Accuracy: {val_acc:.4f} ')

        evaluation_results['train_loss'].append(train_loss)
        evaluation_results['train_acc'].append(train_acc)
        #evaluation_results['val_loss'].append(val_loss)
        #evaluation_results['val_acc'].append(val_acc)
        
    return evaluation_results
    

In [71]:
epochs = 20

model_evaluation_results = train_model(
    epochs=epochs, 
    model=model, 
    optimizer=optimizer, 
    loss_fn=loss_fn,
    train_dataloader=train_loader, 
    val_dataloader=val_loader,
    )
#print(model_evaluation_results)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1
----------------------


 10%|█         | 2/20 [00:00<00:03,  4.89it/s]

Train Loss: 0.8534 | Train Accuracy: 0.6072 
Epoch 2
----------------------
Train Loss: 0.8460 | Train Accuracy: 0.6171 
Epoch 3
----------------------


 20%|██        | 4/20 [00:00<00:02,  7.25it/s]

...

Train Loss: 0.6624 | Train Accuracy: 0.7812 
Epoch 16
----------------------
Train Loss: 0.6471 | Train Accuracy: 0.7823 
Epoch 17
----------------------


 90%|█████████ | 18/20 [00:02<00:00,  9.47it/s]

Train Loss: 0.6336 | Train Accuracy: 0.7836 
Epoch 18
----------------------
Train Loss: 0.6216 | Train Accuracy: 0.7855 
Epoch 19
----------------------


100%|██████████| 20/20 [00:02<00:00,  8.43it/s]

Train Loss: 0.6146 | Train Accuracy: 0.7854 
Epoch 20
----------------------
Train Loss: 0.6035 | Train Accuracy: 0.7940 


In [ ]:
# Save the Model
from pathlib import Path

MODEL_PATH = Path('models')
MODEL_PATH.mkdir(parents=True,
                 exist_ok=True)
MODEL_NAME = 'nn_model_v0.pth'
MODEL_SAVE_PATH = MODEL_PATH/MODEL_NAME

# Save
print(f'Saving the model to: {MODEL_SAVE_PATH}')
torch.save(obj=model.state_dict(),f=MODEL_SAVE_PATH)

In [ ]:
import matplotlib.pyplot as plt
def plot_evaluation_metrics(evaluation_results):
    epochs = list(range(1, len(evaluation_results['train_loss']) + 1))

    plt.figure(figsize=(14, 5))

    # Plot training and test loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, evaluation_results['train_loss'], label='Train Loss')
    plt.plot(epochs, evaluation_results['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Over Epochs')
    plt.xticks(epochs)  # Ensure x-axis ticks are integers
    plt.legend()

    # Plot training and test accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, [acc * 100 for acc in evaluation_results['train_acc']], label='Train Accuracy (%)')
    plt.plot(epochs, [acc * 100 for acc in evaluation_results['val_acc']], label='Validation Accuracy (%)')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.title('Accuracy Over Epochs')
    plt.xticks(epochs)  # Ensure x-axis ticks are integers
    plt.legend()

    plt.show()



In [ ]:
plot_evaluation_metrics(model_evaluation_results)

In [ ]:
print(f'Model Accuracy: {model_evaluation_results['val_acc'][-1]}')

In [ ]:
torch.manual_seed(42)
def make_predictions(model, dataloader):
    device = 'cpu'
    y_preds = []
    model.eval()
    model.to(device)
    
    with torch.inference_mode():
        for X  in tqdm(dataloader, desc='Making Predictions...'):
            X = X[0]
            X = X.to(device)         
            y_logit = model(X)
            
            y_prob = torch.sigmoid(y_logit)
            
            y_labels = (y_prob >= 0.5).int()
            
            y_preds.append(y_labels.cpu())
                
    y_pred_tensor = torch.cat(y_preds) # Concats multiple tensors to one
    
    return y_pred_tensor

In [ ]:
val_preds = make_predictions(model, val_loader).squeeze().numpy()
(val_preds == df_val['Survived']).sum()


In [ ]:
len(df_val)

In [ ]:
torch.manual_seed(42)
df_test = pd.read_csv('test.csv',index_col='PassengerId')
df_test = preprocess_dataset(df_test)
df_test[numerical_columns] = scaler.transform(df_test[numerical_columns])
df_test

In [ ]:
torch.manual_seed(42)
X_test = torch.tensor(df_test.values, dtype=torch.float32)
test_dataset = TensorDataset(X_test)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)  # Batch size can be adjusted

In [ ]:
preds = make_predictions(model, test_loader).squeeze().numpy()
preds[0:10]


In [ ]:
df_preds = pd.DataFrame(data={'Survived':preds}, index=df_test.index)

In [ ]:
df_preds.to_csv('predictions.csv')